<a href="https://colab.research.google.com/github/thotran2015/COVID19_Causality/blob/master/Copy_of_covariate_adjustment_network_effect_interaction_terms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
# !pip install -U plotly
# import plotly.express as px
from urllib.request import urlopen

import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import pandas
import time
import io
import json

#You will need to mount the drive to use the relevant datasets in the shared 'Data' folder
from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def load_data(ver=2, t_type='unacast'):
  if ver == 2:
    df_X = pd.read_csv('/content/drive/My Drive/COVID19 Data Exploration /Data/all_X_ver2.csv')
    del df_X['Unnamed: 0']
    df_Y = pd.read_csv('/content/drive/My Drive/COVID19 Data Exploration /Data/case_diff_Y_v2.csv')
    del df_Y['Unnamed: 0']
  elif ver == 1:
    df_X = pd.read_csv('/content/drive/My Drive/COVID19 Data Exploration /Data/all_X.csv')
    del df_X['Unnamed: 0']
    df_Y = pd.read_csv('/content/drive/My Drive/COVID19 Data Exploration /Data/case_diff_Y.csv')
    del df_Y['Unnamed: 0']

  if t_type == 'unacast':
    df_T = pd.read_csv('/content/drive/My Drive/COVID19 Data Exploration /Data/unacast_T.csv')
    del df_T['Unnamed: 0']
  elif t_type == 'safegraph':
    df_T = pd.read_csv('/content/drive/My Drive/COVID19 Data Exploration /Data/safegraph_data.csv')
  df_T = df_T.rename(columns = {'FIPS-Code':'fips', 'Social_Distance_Change':'score'})

  df_X['fips'] = df_X['fips'].astype(float).astype(int)
  df_Y['fips'] = df_Y['fips'].astype(float).astype(int)
  df_T['fips'] = df_T['fips'].astype(float).astype(int)

  return df_X, df_Y, df_T

In [0]:
import math
def construct_dataset(df_X, df_Y, df_T, Y_mode='adj_increase', population_adjustment_factor=1000., num_task=3, t_type='safegraph', \
                      average_y=True):
  """
  Returns dataset as a dict {}
  Key: FIPS
  Value: Tuple (X, Y, T) where X, Y, T are arrays
  """
  data = {}

  score = 'score' if t_type == 'safegraph' else 'unacast_score'
  for idx, row in df_X.iterrows():
    if len(df_Y.loc[df_Y['fips']==row['fips']]) == 0 or len(df_T.loc[df_T['fips']==row['fips']]) == 0 :
      continue
    if math.isnan(df_T.loc[df_T['fips']==row['fips']][score].values[0]):
      continue
    id = row['fips']
    x = row.values[3:]
    relevant_x = []  
    # relevant_x.append(x[0])         # ratio of male
    # relevant_x.append(sum(x[2:13])) # ratio of people under 60
    relevant_x.append(x[21]) # ratio (%) of african american population
    relevant_x.append(x[25]) # ratio of hispanic population
    # for k in x[20:26]:
    #  relevant_x.append(k) # race distribution 
    for i, k in enumerate(x[30:37]):
      if i==3:
        continue
      relevant_x.append(k) # 31: poverty, 32: unemployment, 33: median income, 34: pop density, 35: total test per capita, 36: positive test rate, 37: democrat support rate
    relevant_x.append(float(x[39]) / float(x[26])) # current confirmed case / capita
    relevant_x.append(float(x[40]) / float(x[26])) # current confirmed death / capita

    if Y_mode == 'adj_increase':
      relevant_x.append(x[42]) # per capita increase during March 23-28
    else:
      relevant_x.append(x[41])

    y1_key = 'Y1_' + Y_mode
    y1 = df_Y.loc[df_Y['fips']==id][y1_key].values[0] / 7.0

    y2_key = 'Y2_' + Y_mode
    y2 = df_Y.loc[df_Y['fips']==id][y2_key].values[0] / 7.0

    y3_key = 'Y3_' + Y_mode
    y3 = df_Y.loc[df_Y['fips']==id][y3_key].values[0] / 7.0

    # average daily case within 5, 10, 15 days
    if average_y and Y_mode == 'adj_increase':
      y1 = y1
      y2 = (y1+y2) / 2.0
      y3 = (y3 + 2.0*y2) / 3.0

    if Y_mode == 'adj_increase':
      y1 *= population_adjustment_factor
      y2 *= population_adjustment_factor
      y3 *= population_adjustment_factor

    y = [y1, y2, y3] if num_task == 3 else ([y1, y2] if num_task == 2 else [y1])

    t = df_T.loc[df_T['fips']==id][score].values[0]
    # t = 1 if t > 4 else 0 # binarize T (A, B --> 1, C, D, F --> 0)

    id_key = "{:05}".format(id)
    data[id_key] = (relevant_x, y, t)
  
  return data


In [0]:
df_X, df_Y, df_T = load_data(ver=2, t_type='unacast')
data = construct_dataset(df_X, df_Y, df_T, Y_mode='adj_increase', num_task=3, t_type='unacast')

print(len(data))
print(data['25025'][0]) # Suffolk County, Massachusetts
print(data['25025'][1]) # Suffolk County, Massachusetts
print(data['25025'][2]) # Suffolk County, Massachusetts

2524
[0.24667761739828456, 0.23381050774727105, 0.44799999999999995, 0.1649348654447434, 0.03, 13859.478078, 0.05627073538980396, 0.2002966098421943, 0.0020117633650954102, 1.8581558175142334e-05, 0.0015947118261191904]
[0.47606786072802665, 0.4851307427351671, 0.5294974004041098]
8.0


In [0]:
# Split Train, Val, Test
import random
# test = {'0': 0, '1': 1, '2':2, '3':3}
# keys = list(test.keys())
# shuffledKeys = random.sample(keys, 4)
# print(shuffledKeys)

def generate_split(keys, val_size=0.1, test_size=0.2):
  if val_size + test_size >= 1.0:
    raise ValueError("Sum of the validation size and the test size should be less than 1")
  shuffled_keys = random.sample(keys, len(keys))

  total_len = len(keys)
  train_keys = shuffled_keys[:int(total_len*(1-val_size-test_size))]
  val_keys = shuffled_keys[int(total_len*(1-val_size-test_size)):int(total_len*(1-test_size))]
  test_keys = shuffled_keys[int(total_len*(1-test_size)):]

  return train_keys, val_keys, test_keys

train_keys, val_keys, test_keys = generate_split(list(data.keys()))
print(len(train_keys), len(val_keys), len(test_keys))

1766 253 505


In [0]:
def construct_split_dataset(data, train_keys, val_keys, test_keys, censor_by='all'):
  XT_train = []
  XT_val = []
  XT_test = []

  Y_train = []
  Y_val= []
  Y_test = []

  final_train_keys = []
  for key in train_keys:
    x, y, t = data[key]
    y = np.asarray(y)
    x = np.asarray(x)
    last_three_x = x[len(x)-3:]
    flag = (last_three_x[0]> 0 and last_three_x[1]>0 and last_three_x[2]>0) if censor_by == 'all' else (last_three_x[0]> 0 and last_three_x[2]>0)
    if not flag:
      continue
    
    feature_interact = t * x
    feature = np.insert(x, x.shape[0], t)
    feature = np.concatenate((feature, feature_interact))
    if np.isnan(feature.sum()):
      continue
    if np.isnan(y.sum()):
      continue
    if not (y[0] > 0 and y[1] > 0 and y[2] > 0 ):
      continue

    XT_train.append(feature)
    Y_train.append(y)
    final_train_keys.append(key)

  final_val_keys = []
  for key in val_keys:
    x, y, t = data[key]
    y = np.asarray(y)
    x = np.asarray(x)
    last_three_x = x[len(x)-3:]
    flag = (last_three_x[0]> 0 and last_three_x[1]>0 and last_three_x[2]>0) if censor_by == 'all' else (last_three_x[0]> 0 and last_three_x[2]>0)
    if not flag:
      continue

    feature_interact = t * x
    feature = np.insert(x, x.shape[0], t)
    feature = np.concatenate((feature, feature_interact))
    if np.isnan(feature.sum()):
      continue
    if np.isnan(y.sum()):
      continue
    if not (y[0] > 0 and y[1] > 0 and y[2] > 0):
      continue
    XT_val.append(feature)
    Y_val.append(y)
    final_val_keys.append(key)

  final_test_keys = []
  for key in test_keys:
    x, y, t = data[key]
    y = np.asarray(y)
    x = np.asarray(x)
    last_three_x = x[len(x)-3:]
    flag = (last_three_x[0]> 0 and last_three_x[1]>0 and last_three_x[2]>0) if censor_by == 'all' else (last_three_x[0]> 0 and last_three_x[2]>0)
    if not flag:
      continue

    feature_interact = t * x
    feature = np.insert(x, x.shape[0], t)
    feature = np.concatenate((feature, feature_interact))
    if np.isnan(feature.sum()):
      continue
    if np.isnan(y.sum()):
      continue
    if not (y[0] > 0 and y[1] > 0 and y[2] > 0):
      continue
    XT_test.append(feature)
    Y_test.append(y)
    final_test_keys.append(key)

  XT_train = np.asarray(XT_train)
  XT_val = np.asarray(XT_val)
  XT_test = np.asarray(XT_test)
  Y_train = np.asarray(Y_train)
  Y_val = np.asarray(Y_val)
  Y_test = np.asarray(Y_test)
  return XT_train, XT_val, XT_test, Y_train, Y_val, Y_test, final_train_keys, final_val_keys, final_test_keys

In [0]:
# Train the MultiTaskLasso
from sklearn import linear_model, metrics, multioutput, neighbors

def train_model(XT_train, Y_train, XT_val, Y_val, scaler, y_scaler, alpha, l1_ratio, y_scale=False, model_type='linear', n_neighbors=20):
  if model_type == 'linear':
    # model = linear_model.MultiTaskLasso(alpha=alpha, max_iter=100000)
    model = linear_model.MultiTaskElasticNet(alpha=alpha, l1_ratio=l1_ratio)
    # model = multioutput.RegressorChain(linear_model.Ridge(alpha=alpha, max_iter=100000))
  elif model_type == 'knn':
    basemodel = neighbors.KNeighborsRegressor(n_neighbors=n_neighbors, weights='distance')
    model = multioutput.RegressorChain(basemodel)
  XT_train_transformed = scaler.transform(XT_train)
  # print(XT_train_transformed.mean(axis=0))
  if y_scale:
    Y_train_ = y_scaler.transform(Y_train)
  else:
    Y_train_ = Y_train

  model.fit(XT_train_transformed, Y_train_)

  pred = model.predict(XT_train_transformed)
  mse = metrics.mean_squared_error(Y_train_, pred)
  print("Mean Squred Error for the Train Set: {:.4f}".format(mse))
  print("R^2 for this model (closer to 1, the better): {:.4f}".format(model.score(XT_train_transformed, Y_train_)))

  XT_val_transformed = scaler.transform(XT_val)
  if y_scale:
    Y_val_ = y_scaler.transform(Y_val)
  else:
    Y_val_ = Y_val
  pred = model.predict(XT_val_transformed)
  mse = metrics.mean_squared_error(Y_val_, pred)
  print("Mean Squred Error for the Validation Set: {:.4f}".format(mse))
  print("R^2 for this model (closer to 1, the better): {:.4f}".format(model.score(XT_val_transformed, Y_val_)))
  # print("Printing predictions on the validataion set")
  # print("Index , [Ground Truth], [Predicted]")
  # Y_val_ = y_scaler.inverse_transform(Y_val_)
  # pred = y_scaler.inverse_transform(pred)
  # for idx, (y, p) in enumerate(zip(Y_val_, pred)):
  #   print(idx, y, p)

  return model

def test_model(model, XT, Y, scaler, y_scaler, y_scale=True):
  XT_transformed = scaler.transform(XT)
  if y_scale:
    Y_ = y_scaler.transform(Y)
  else:
    Y_ = y
  pred = model.predict(XT_transformed)
  mse = metrics.mean_squared_error(Y_, pred)
  print("Mean Squred Error for the Test Set: {:.4f}".format(mse))
  print("R^2 for this model (closer to 1, the better): {:.4f}".format(model.score(XT_transformed, Y_)))
  print("Printing predictions on the validataion set")
  print("Index , [Ground Truth], [Predicted]")
  Y_val_ = y_scaler.inverse_transform(Y_)
  pred = y_scaler.inverse_transform(pred)
  for idx, (y, p) in enumerate(zip(Y_val_, pred)):
    print(idx, y, p)


Including Networking Effect with Simple Markov Assumption

In [0]:
import csv
PATH = '/content/drive/My Drive/COVID19 Data Exploration /Data/county_adjacency.csv'
county_df = pd.read_csv(PATH, header=None)
county_df.head()

,0,1
0,1001.0,1001
1,NaN,1021
2,NaN,1047
3,NaN,1051
4,NaN,1085


In [0]:
county_adjacency = {}
prev_fips = float('nan')
for idx, row in county_df.iterrows():
  fips = row[0]
  neighbor = row[1]
  if math.isnan(fips):
    fips = prev_fips
  else:
    prev_fips = fips
  if int(fips) in county_adjacency.keys():
    county_adjacency[int(fips)].append(int(neighbor))
  else:
    county_adjacency[int(fips)] = [int(neighbor)]

print(county_adjacency[1001])


[1001, 1021, 1047, 1051, 1085, 1101]


In [0]:
def check_data_validity(x, y, t, censor_by):
  last_three_x = x[len(x)-3:]
  flag = (last_three_x[0]> 0 and last_three_x[1]>0 and last_three_x[2]>0) if censor_by == 'all' else (last_three_x[0]> 0 and last_three_x[2]>0)
  if not flag:
    return False
  feature = np.insert(x, len(x), t)
  if np.isnan(feature.sum()):
    return False
  if np.isnan(y.sum()):
    return False
  if not (y[0] > 0 and y[1] > 0 and y[2] > 0):
    return False

  return True

def process_data_with_adjacency(data, keys, adjacency, censor_by='all', neighbor_strength=1.0):
  XT = []
  Y = []
  final_keys = []

  for key in keys:
    x, y, t = data[key]
    y = np.asarray(y)
    x = np.asarray(x)
    valid = check_data_validity(x, y, t, censor_by)
    if valid == False:
      continue
    xt_interact = t * x
    xt = np.insert(x, x.shape[0], t)
    xt = np.concatenate((xt, xt_interact))
    final_keys.append(key)

    adj_xt= []
    if int(key) in adjacency.keys():
      neighbors = adjacency[int(key)]
      for n in neighbors:
        # Exclude itself
        if int(n) == int(key):
          continue
        if "{:05}".format(n) in data.keys():
          a_x, a_y, a_t = data["{:05}".format(n)]
          a_y = np.asarray(a_y)
          a_x = np.asarray(a_x)
          a_valid = check_data_validity(a_x, a_y, a_t, censor_by)
          if a_valid:
            a_xt_interact = a_t * a_x
            a_xt = np.insert(a_x, a_x.shape[0], a_t)
            a_xt = np.concatenate((a_xt, a_xt_interact))
            adj_xt.append(a_xt)
      
    if len(adj_xt) > 0:
      # print(len(adj_xt))
      adj_xt = np.asarray(adj_xt)
      adj_effect = adj_xt.mean(axis=0)
      xt = xt + neighbor_strength * adj_effect
    else:
      xt = (1 + neighbor_strength) * xt
    
    XT.append(xt)
    Y.append(y)

  return XT, Y, final_keys


In [0]:
neighbor_XT_train, neighbor_Y_train, neighbor_final_keys = process_data_with_adjacency(data, train_keys, county_adjacency, \
                                                                                       censor_by='death-only', neighbor_strength=1.0)


In [0]:
neighbor_XT_val, neighbor_Y_val, neighbor_final_val = process_data_with_adjacency(data, val_keys, county_adjacency, censor_by='death-only', neighbor_strength=1.0)
neighbor_XT_test, neighbor_Y_test, neighbor_final_test = process_data_with_adjacency(data, test_keys, county_adjacency, censor_by='death-only', neighbor_strength=1.0)

In [0]:
# Standaradization for XT
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(neighbor_XT_train)
y_scaler = preprocessing.StandardScaler().fit(neighbor_Y_train)

print(scaler.mean_)
print(y_scaler.mean_)

for alpha in [0, 1e-3, 2e-3, 5e-3, 1e-2, 2e-2, 5e-2]:
  for l1 in [0, 0.01, 0.1, 0.2, 0.5]:
    print(alpha, l1)
    model = train_model(neighbor_XT_train, neighbor_Y_train, neighbor_XT_val, neighbor_Y_val, scaler, y_scaler, alpha, l1, True)
# coef = model.coef_
# print(coef)
# for i in range(3):
#   c = coef[i]
#   print(c.shape)
#   print("------------------------------------")
#   print("{}th Task".format(i))
#   # print("Sex (Male ratio): {:.5f}".format(c[0]))
#   # print("Age (<60 ratio) : {:.5f}".format(c[1]))
#   # print("Race - White    : {:.5f}".format(c[0]))
#   print("Race - Black    : {:.5f}".format(c[0]))
#   # print("Race - Native   : {:.5f}".format(c[4]))
#   # print("Race - Pacific  : {:.5f}".format(c[5]))
#   # print("Race - Asian    : {:.5f}".format(c[6]))
#   print("Race - Hispanic : {:.5f}".format(c[1]))
#   print("Bachelor degree : {:.5f}".format(c[2]))
#   print("Poverty         : {:.5f}".format(c[3]))
#   print("Unemployment    : {:.5f}".format(c[4]))
#   # print("Median Income   : {:.5f}".format(c[7]))
#   print("Pop density     : {:.5f}".format(c[5]))
#   print("Test / capita   : {:.5f}".format(c[6]))
#   print("Postive test    : {:.5f}".format(c[7]))
#   # print("Politic - Dem   : {:.5f}".format(c[8]))
#   print("Covid - Case    : {:.5f}".format(c[8]))
#   print("Covid - Death   : {:.5f}".format(c[9]))
#   print("Covid - Increase: {:.5f}".format(c[10]))
#   print("TREATMENT       : {:.5f}".format(c[11]))

[2.52562182e-01 2.04201933e-01 5.29347550e-01 2.68269441e-01
 8.08688241e-02 8.71789616e+02 5.36836253e-02 2.50602717e-01
 8.10728174e-04 2.10656293e-05 5.70731892e-04 1.01571948e+01
 1.27620397e+00 1.14217341e+00 2.96033295e+00 1.31245472e+00
 4.02823041e-01 5.87638760e+03 2.74701436e-01 1.33224782e+00
 4.76359769e-03 1.13581458e-04 3.32999695e-03]
[0.06073016 0.06413753 0.06496601]
0 0


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:1790: UserWarning: Coordinate descent with l1_reg=0 may lead to unexpected results and is discouraged.
  check_random_state(self.random_state), random)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:1790: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 779.0012256165375, tolerance: 0.2706
  check_random_state(self.random_state), random)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "mul

Mean Squred Error for the Train Set: 0.5758
R^2 for this model (closer to 1, the better): 0.4242
Mean Squred Error for the Validation Set: 0.1563
R^2 for this model (closer to 1, the better): 0.5298
0 0.01


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:1790: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 779.0012256165375, tolerance: 0.2706
  check_random_state(self.random_state), random)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_av

Mean Squred Error for the Train Set: 0.5758
R^2 for this model (closer to 1, the better): 0.4242
Mean Squred Error for the Validation Set: 0.1563
R^2 for this model (closer to 1, the better): 0.5298
0 0.1


...

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multi

Mean Squred Error for the Train Set: 0.5842
R^2 for this model (closer to 1, the better): 0.4158
Mean Squred Error for the Validation Set: 0.1393
R^2 for this model (closer to 1, the better): 0.5810
0.02 0.01
Mean Squred Error for the Train Set: 0.5843
R^2 for this model (closer to 1, the better): 0.4157
Mean Squred Error for the Validation Set: 0.1392
R^2 for this model (closer to 1, the better): 0.5812
0.02 0.1
Mean Squred Error for the Train Set: 0.5859
R^2 for this model (closer to 1, the better): 0.4141
Mean Squred Error for the Validation Set: 0.1390
R^2 for this model (closer to 1, the better): 0.5819
0.02 0.2
Mean Squred Error for the Train Set: 0.5873
R^2 for this model (closer to 1, the better): 0.4127
Mean Squred Error for the Validation Set: 0.1396
R^2 for this model (closer to 1, the better): 0.5802
0.02 0.5
Mean Squred Error for the Train Set: 0.5914
R^2 for this model (closer to 1, the better): 0.4086
Mean Squred Error for the Validation Set: 0.1434
R^2 for this model (c

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multi

Mean Squred Error for the Train Set: 0.5899
R^2 for this model (closer to 1, the better): 0.4101
Mean Squred Error for the Validation Set: 0.1410
R^2 for this model (closer to 1, the better): 0.5758
0.05 0.01
Mean Squred Error for the Train Set: 0.5901
R^2 for this model (closer to 1, the better): 0.4099
Mean Squred Error for the Validation Set: 0.1411
R^2 for this model (closer to 1, the better): 0.5755
0.05 0.1
Mean Squred Error for the Train Set: 0.5922
R^2 for this model (closer to 1, the better): 0.4078
Mean Squred Error for the Validation Set: 0.1427
R^2 for this model (closer to 1, the better): 0.5708
0.05 0.2
Mean Squred Error for the Train Set: 0.5943
R^2 for this model (closer to 1, the better): 0.4057
Mean Squred Error for the Validation Set: 0.1446
R^2 for this model (closer to 1, the better): 0.5649
0.05 0.5
Mean Squred Error for the Train Set: 0.5973
R^2 for this model (closer to 1, the better): 0.4027
Mean Squred Error for the Validation Set: 0.1446
R^2 for this model (c

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multi

In [0]:
model = train_model(neighbor_XT_train, neighbor_Y_train, neighbor_XT_val, neighbor_Y_val, scaler, y_scaler, 0.01, 0.5, True)
test_model(model, neighbor_XT_test, neighbor_Y_test, scaler, y_scaler, y_scale=True)

Mean Squred Error for the Train Set: 0.5860
R^2 for this model (closer to 1, the better): 0.4140
Mean Squred Error for the Validation Set: 0.1379
R^2 for this model (closer to 1, the better): 0.5850
Mean Squred Error for the Test Set: 1.5722
R^2 for this model (closer to 1, the better): 0.1828
Printing predictions on the validataion set
Index , [Ground Truth], [Predicted]
0 [0.01373692 0.01717116 0.01144744] [0.0568573  0.0695402  0.06633932]
1 [0.02676079 0.01873256 0.0472774 ] [0.03870748 0.04312071 0.05975519]
2 [0.05606565 0.05700008 0.05513122] [0.04176896 0.05570216 0.05977933]
3 [0.08012928 0.08680671 0.06974215] [0.10191165 0.10786155 0.10009805]
4 [0.20720069 0.16699757 0.15978162] [0.11250364 0.10486645 0.10493825]
5 [0.01449021 0.01539585 0.01569772] [0.02857892 0.03727012 0.03525465]
6 [0.04493069 0.04867492 0.05491529] [0.04987341 0.06556732 0.06761382]
7 [0.01676216 0.02346702 0.0391117 ] [0.06878151 0.07967295 0.08291936]
8 [0.03022939 0.02441605 0.01782759] [0.06163912 

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multi

266 [0.00218778 0.00355514 0.00382862] [-0.00036645  0.01026156  0.01165265]
267 [0.03219227 0.03471581 0.03414745] [0.02345106 0.02857508 0.03194896]


Analysis: CATE

In [0]:
def cate_measurement(model, scaler, y_scaler, fips, data, df_X, adjacency, neighbor_t=False, censor_by='death-only', neighbor_strength=1.0):
  # Return the expected Y's when T is different (1~9 for Unacast)
  STNAME = df_X.loc[df_X['fips']==int(fips)]['STNAME'].values[0]
  CTYNAME = df_X.loc[df_X['fips']==int(fips)]['CTYNAME'].values[0]
  pop = df_X.loc[df_X['fips']==int(fips)]['total_pop'].values[0]

  print("Fips [{}] {} - {}".format(fips, STNAME, CTYNAME))
  print("Total population: {}".format(pop))
  x, y, t = data[fips]
  y = np.asarray(y)
  x = np.asarray(x)
  print("Original T: {}".format(t))
  print("Original Y1, Y2, Y3: {:.4f}, {:.4f}, {:.4f}".format(y[0]*100, y[1]*100, y[2]*100))

  for tt in range(1, 10):
    feature_interact = tt * x
    feature = np.insert(x, x.shape[0], tt)
    feature = np.concatenate((feature, feature_interact))
    adj_xt= []
    if int(fips) in adjacency.keys():
      neighbors = adjacency[int(fips)]
      for n in neighbors:
        # Exclude itself
        if int(n) == int(fips):
          continue
        if "{:05}".format(n) in data.keys():
          a_x, a_y, a_t = data["{:05}".format(n)]
          if neighbor_t:
            a_t = tt
          a_y = np.asarray(a_y)
          a_x = np.asarray(a_x)
          a_valid = check_data_validity(a_x, a_y, a_t, censor_by)
          if a_valid:
            a_xt_interact = a_t * a_x
            a_xt = np.insert(a_x, a_x.shape[0], a_t)
            a_xt = np.concatenate((a_xt, a_xt_interact))
            adj_xt.append(a_xt)
      
    if len(adj_xt) > 0:
      # print(len(adj_xt))
      adj_xt = np.asarray(adj_xt)
      adj_effect = adj_xt.mean(axis=0)
      feature = feature + neighbor_strength * adj_effect
    else:
      feature = (1+neighbor_strength) * feature

    XT = scaler.transform(feature.reshape((1, -1)))
    pred = model.predict(XT)
    pred = y_scaler.inverse_transform(pred)[0]
    print("When T={}, Y's are: {:.4f}, {:.4f}, {:.4f}".format(tt, pred[0]*100, pred[1]*100, pred[2]*100))

    if tt == 1:
      pred_1 = pred
    if tt == 9:
      pred_9 = pred

  return pred_1, pred_9

In [0]:
cate_1 = 0
cate_2 = 0
cate_3 = 0
for key in neighbor_final_test:
  pred_1, pred_9 = cate_measurement(model, scaler, y_scaler, key, data, df_X, county_adjacency, neighbor_t=True)
  print("----------------------------------------------------------")
  cate_1 += pred_9[0] - pred_1[0]
  cate_2 += pred_9[1] - pred_1[1]
  cate_3 += pred_9[2] - pred_1[2]

cate_1 /= float(len(neighbor_final_test))
cate_2 /= float(len(neighbor_final_test))
cate_3 /= float(len(neighbor_final_test))

print(cate_1*100, cate_2*100, cate_3*100)

Fips [18119] Indiana - Owen County
Total population: 20845
Original T: 5.0
Original Y1, Y2, Y3: 1.3737, 1.7171, 1.1447
When T=1, Y's are: 5.1620, 7.5063, 7.6875
When T=2, Y's are: 5.3598, 7.4361, 7.5035
When T=3, Y's are: 5.5576, 7.3658, 7.3196
When T=4, Y's are: 5.7553, 7.2956, 7.1356
When T=5, Y's are: 5.9531, 7.2253, 6.9517
When T=6, Y's are: 6.1509, 7.1551, 6.7677
When T=7, Y's are: 6.3486, 7.0849, 6.5837
When T=8, Y's are: 6.5464, 7.0146, 6.3998
When T=9, Y's are: 6.7442, 6.9444, 6.2158
----------------------------------------------------------
Fips [36035] New York - Fulton County
Total population: 53591
Original T: 2.5
Original Y1, Y2, Y3: 2.6761, 1.8733, 4.7277
When T=1, Y's are: -1.4460, -0.5037, 0.9181
When T=2, Y's are: 0.8800, 1.5599, 3.0336
When T=3, Y's are: 3.2059, 3.6235, 5.1491
When T=4, Y's are: 5.5319, 5.6871, 7.2646
When T=5, Y's are: 7.8578, 7.7507, 9.3802
When T=6, Y's are: 10.1838, 9.8143, 11.4957
When T=7, Y's are: 12.5098, 11.8779, 13.6112
When T=8, Y's are: 14

In [0]:
cate_measurement(model, scaler, y_scaler, '25025', data, df_X, county_adjacency, neighbor_t=True)
cate_measurement(model, scaler, y_scaler, '25025', data, df_X, county_adjacency, neighbor_t=False)

Fips [25025] Massachusetts - Suffolk County
Total population: 807252
Original T: 8.0
Original Y1, Y2, Y3: 47.6068, 48.5131, 52.9497
When T=1, Y's are: 19.3909, 22.8273, 30.2337
When T=2, Y's are: 21.9624, 24.9170, 31.2872
When T=3, Y's are: 24.5339, 27.0067, 32.3407
When T=4, Y's are: 27.1054, 29.0964, 33.3942
When T=5, Y's are: 29.6769, 31.1860, 34.4477
When T=6, Y's are: 32.2484, 33.2757, 35.5012
When T=7, Y's are: 34.8199, 35.3654, 36.5547
When T=8, Y's are: 37.3914, 37.4551, 37.6082
When T=9, Y's are: 39.9629, 39.5448, 38.6617
Fips [25025] Massachusetts - Suffolk County
Total population: 807252
Original T: 8.0
Original Y1, Y2, Y3: 47.6068, 48.5131, 52.9497
When T=1, Y's are: 24.1595, 30.2458, 36.5243
When T=2, Y's are: 25.9609, 31.1697, 36.5653
When T=3, Y's are: 27.7623, 32.0937, 36.6064
When T=4, Y's are: 29.5636, 33.0176, 36.6474
When T=5, Y's are: 31.3650, 33.9415, 36.6884
When T=6, Y's are: 33.1664, 34.8655, 36.7295
When T=7, Y's are: 34.9677, 35.7894, 36.7705
When T=8, Y's ar

(array([0.24159537, 0.30245821, 0.36524276]),
 array([0.38570469, 0.37637244, 0.36852616]))